<a href="https://www.kaggle.com/code/dhruvtiwari/easy-xgbmodel?scriptVersionId=131557392" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Easy to understand XGBoost model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Data Preprocessing

In [ ]:
X = pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv')
y = X['Age']
X.drop('Age',inplace=True,axis=1)
test = pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv')

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
X.describe()

In [ ]:
X.isnull().sum()

There are no null values which is good. The Id column is useless so drop it

In [ ]:
useless_cols = ['id']
X = X.drop(useless_cols,axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

Ordinal Encoder to convert Sex values I,M,F to 0,1,2 which are numerical

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

Enc = OrdinalEncoder()
X_train['Sex'] = Enc.fit_transform(X_train[['Sex']]) 
X_valid['Sex'] = Enc.transform(X_valid[['Sex']]) 
test['Sex'] = Enc.transform(test[['Sex']])


In [ ]:
X_train

Training XGBoost model with some normal hyperparameters

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=1500,learning_rate=0.1,n_jobs=4,max_depth=4)
model.fit(X_train,y_train)

In [ ]:
predictions = model.predict(X_valid)

Scoring using MAE

In [ ]:
from sklearn.metrics import mean_squared_error
score = mean_squared_error(predictions,y_valid)

In [ ]:
score

Preping the submission DataFrame

In [ ]:
test_id = test['id']
test.drop('id',axis=1,inplace=True)

preds =pd.DataFrame( model.predict(test),columns=['Age'])

In [ ]:
test_id = pd.DataFrame(test_id)

In [ ]:
df = pd.concat([test_id['id'],preds['Age']],axis=1)

In [ ]:
final_df.set_index('id')

In [ ]:
final_df.to_csv('submission.csv')